# ORÁCULO

http://www.lotoideas.com/primitiva-resultados-historicos-de-todos-los-sorteos/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

import time
import operator

Using TensorFlow backend.


In [ ]:
#Muestra una grafica de la evolucion del error de training y validacion
def grafica(hist, offset0 = 0, offset1 = None):
    loss = hist.history['loss'][offset0:offset1]
    val_loss = hist.history['val_loss'][offset0:offset1]
    epochs = range(offset0 + 1, len(loss) + offset0 + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
#Imprime todo el array sin elipsis
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
def print_time(time_ini):
    sec = round(time.time() - time_ini)
    min = 0
    if(sec > 60):
        min = sec // 60
        sec = sec & 60
    print("TIME: ", min, "m  ", sec, "s")

def ini_time():
    print("Procesando... ", time.ctime())
    return time.time()

In [ ]:
#Cargar datos
datos = pd.read_csv('Primitiva_2013_2018.csv')

datos = datos.drop(labels=['comp'], axis=1)

print(datos.head())
print()
#Poner los datos mas recientes al final...
datos = datos.reindex(index=datos.index[::-1])
print(datos.head())
print()

max = datos.max(1).max()
min = datos.min(1).min()
print("MAX: ", max)#49
print("MIN: ", min)# 1
print("DIM: ", datos.shape)
features = max - min +1

In [ ]:
#Convierte los numeros enteros premiados en un array binario
def vectorize(sequences, dimension=features):
    res = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        res[i, sequence-min] = 1
    return res

def devectorize(valor):
    res = []
    for i in range(len(valor)):
        if(valor[i] > 0):
            res.append(i+min)
    return res
#print(devectorize(np.array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0])), "\n")

numeros = datos.drop(labels=['fecha'], axis=1)
print("----------------------------------------\n")
print(numeros.head())
print("----------------------------------------\n")
print(np.array(numeros))
print("----------------------------------------\n")
num_hot = vectorize(np.array(numeros))
print(devectorize(num_hot[0]))
print(devectorize(num_hot[1]))
print(devectorize(num_hot[2]))
print("...")

In [ ]:
#a = np.array(numeros)
#b = np.zeros((6, numeros.size()))
#b[np.arange(3), a] = 1

In [ ]:
#Divide los datos en secuencias que el modelo tendrá que aprender
seq_len = 100
secuencias = []
print(len(num_hot), seq_len)
for index in range(len(num_hot) - seq_len):
    #print(index, (num_hot[index : index + seq_len]))
    secuencias.append(num_hot[index : index + seq_len])
secuencias = np.array(secuencias)
###
print(devectorize(num_hot[0]))
print(devectorize(secuencias[0][0]))
print()
print(devectorize(num_hot[seq_len-1]))
print(devectorize(secuencias[0][seq_len-1]))
print()
print(devectorize(num_hot[1]))
print(devectorize(secuencias[1][0]))
print()
print(devectorize(num_hot[1+seq_len-1]))
print(devectorize(secuencias[1][seq_len-1]))
print()

In [ ]:
print(devectorize(secuencias[0][0]))
print("...")

In [ ]:
#Divide los datos en Training y Test
print(secuencias.shape)
row = round(0.9 * secuencias.shape[0])
train = secuencias[:int(row), :]
print(train.shape)
print(devectorize(train[0][0]))
print(devectorize(secuencias[0][0]))


#np.random.shuffle(train)

x_train = train[:, :-1]#Todos menos el ultimo valor, que es el que hay que predecir
print(x_train.shape)
print(x_train[0][0])

y_train = train[:, -1] #El ultimo valor, que es el que interesa predecir
print(y_train.shape)
x_test = secuencias[int(row):, :-1]
print(x_test.shape)
y_test = secuencias[int(row):, -1]
print(y_test.shape)


In [ ]:
print(x_train.shape,  seq_len)

print()
print(devectorize(num_hot[0]))
print(devectorize(x_train[0][0]))
print()
print(devectorize(num_hot[seq_len-2]))
print(devectorize(x_train[0][seq_len-2]))
print()
print(devectorize(num_hot[seq_len-1]))
print(devectorize(y_train[0]))
print()
print(devectorize(num_hot[x_train.shape[0]+seq_len-2]))
print(devectorize(x_test[0][seq_len-2]))
print()
print(devectorize(num_hot[x_train.shape[0]+seq_len-1]))
print(devectorize(y_test[0]))

# --------------------------------------------------------------------------------------------------------------

model1 ->

4 capas LSTM, 1 densa, epoca=35, 10m4s -> 95% error 




In [ ]:
#Crea la RNN
def build_model():
    model = models.Sequential()
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))
    #model.add(layers.Dropout(0.2))
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))

    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=False))
    #model.add(layers.Dropout(0.2))

    model.add(layers.Dense(units=features, activation='sigmoid'))
    #model.add(layers.Dense(units=features))
    #model.add(layers.Activation("linear"))

    #start = time.time()
    #model.compile(loss="mse", optimizer="rmsprop")
    model.compile(loss="mse", optimizer="rmsprop")
    #print("> Compilation Time : ", round(time.time() - start), " segundos")
    return model

##
def build_model2():
    model = models.Sequential()
    
    model.add(layers.LSTM(
        input_shape=(None, features),
        units=features,
        return_sequences=True))
    
    #model.add(layers.Embedding(features, 32))
    model.add(layers.Bidirectional(layers.LSTM(input_shape=(None, features), units=features, return_sequences=True)))
    model.add(layers.Dense(features, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    return model


In [ ]:
#Entrenar la RNN
batch_size=128
epochs = 200
model = build_model()
timeini = time_ini()
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=0)
print_time(timeini)

In [ ]:
print("Val.Loss: ", hist.history['val_loss'][epochs-1])
grafica(hist)
grafica(hist, 40, 110)

In [ ]:
timeini = ini_time()
best_epoch = 20
model2 = build_model()
hist2 = model2.fit(x_train, y_train, batch_size=batch_size, epochs=best_epoch, validation_split=0.05, verbose=0)
print("Val.Loss: ", hist.history['val_loss'][best_epoch-1])
print_time(timeini)

In [ ]:
grafica(hist2, 10, 35)

In [ ]:
pred = model2.predict(x_test)
print(pred.shape)
print()
print(pred[0])
print()
print("REAL: ", devectorize(y_test[0]))
print()

#TODO: Si el peor de los mejores esta repetido, alguno mejor podria no entrar en la lista
def traduce(pred):
    min_mas_alto = sorted(pred[0], key=float, reverse=True)[5]
    numeros = []
    for i in range(pred.shape[0]):
        numeros.append([])
        for j in range(pred.shape[1]):
            if pred[i][j] >= min_mas_alto and len(numeros[i]) < 6:
                numeros[i].append(j)
    return numeros

numeros_ganadores = traduce(pred)
print("PRED: ", numeros_ganadores[0])

In [ ]:
import math
def calc_error(pred, real):
    error = []
    for i in range(len(pred)):
        error.append(0)
        for j in range(len(pred[i])):
            if(pred[i][j] not in real[i]):
                error[i] += math.floor(100/6)
    return error

error = calc_error(numeros_ganadores, y_test)
print("ERROR: ", error)
err = 0
for x in error:
    err += x
print("\nERROR: ", err/len(error), "% ----------------------------------------")

#pred = np.reshape(pred, (pred.size,))
"""
error = []
for i in range(numeros_ganadores.shape[0]):
    error.append(0)
    for j in range(numeros_ganadores.shape[1]):
        if(pred[i][j] != y_test[i][j]):
            error[i] += 1 """


print("\n\n")

In [ ]:
notas!

In [ ]:
model = Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

In [ ]:
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=1):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
            
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [ ]:
lookback = 1000
step = 1
delay = 1
batch_size = 128

In [ ]:
train_gen = generator(
    num_hot,
    lookback=lookback,
    delay=delay,
    min_index=0,
    max_index=2500,
    shuffle=False,
    step=step,
    batch_size=batch_size)

val_gen = generator(
    num_hot,
    lookback=lookback,
    delay=delay,
    min_index=2501,
    max_index=3502,
    step=step,
    batch_size=batch_size)

test_gen = generator(
    num_hot,
    lookback=lookback,
    delay=delay,
    min_index=3502,
    max_index=None,
    step=step,
    batch_size=batch_size)

val_steps = (3500 - 3001 - lookback)
test_steps = (len(num_hot) - 3700 - lookback)

In [ ]:
#hist = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)
#hist = model.fit_generator(train_gen, steps_per_epoch=500, epochs=20, validation_data=val_gen, validation_steps=val_steps)
